# Kundengruppe Analytiks

In this notebook, we define customer categories based on the data we have in VS4, connect them with some other tables, like the rechnungen, the statistiks and teh newsletter customers, so we have a better view of each customer group. and we then compute their last half-year analytics to see how much revenue each group has brought us.

There are some data quality issues that demanded changes in the regular implementation, namely:

- the Neukunden-1 are customers who are added to our system in the current half-year, and even though, we are analyzing the last half-year, we had these Neukunden to our analytiks too.
- there are some Neukunden-1 customers, whose order date (AUF_ANLAGE) is in 2024, but they are oddly considered Neukunden-1. That is because, the decision to categorize the neukunden-1 group comes down to SYS_ANLAGE (Date of registry in the system) and ERSTKAUF (the first order-date). now this ERSTKAUF column actually has the date that the invoice was created, that can be different, and something way apart from the date that the customer had made their order. that explains this inconsistency.
- this is why, I have tried a few times to define this NK-1 group so that we have a complete list of all those who are Neukunden-1, along with their complete order details.
- the dates are automated. that mean, whenever this code is run, the code looks at the today's date and decide, which Halfyear is the current one, which is the last one, and then makes the analytics, so the pipeline is complete, and it only requires updated data to create an updated analytics.
 

In [ ]:

import pandas as pd
import os
import datetime as dt
from dateutil.relativedelta import relativedelta
from helper import *
from paths import *

## Repetitive setting 
enc = 'cp850'
sep = ';'


In [ ]:
### Defining the dates for the beginning and end of previous and Current HJ, as well as the Number of the Column in KW data
result = get_half_year_info()
last_hj = f'Z{result['number']}'
current_hj = f'Z{result['number']+1}'
prev_start = pd.to_datetime(result['prev_start'])
prev_end = pd.to_datetime(result['prev_end'])
current_start = pd.to_datetime(result['prev_start'] + relativedelta(months=6))
current_end = pd.to_datetime(result['prev_end'] + relativedelta(months=6))

In [ ]:
## Importing all required data
kunden_segments = pd.read_excel(ks_path)
kunden_segment_dict = dict(zip(kunden_segments['Alt'],kunden_segments['Neu']))
kw = pd.read_csv(kw_path, sep=sep,encoding=enc,on_bad_lines='skip')
adresse = pd.read_csv(adresse_path, sep=sep,encoding=enc)
stat = pd.read_csv(stat_path, sep=sep,encoding=enc,usecols=['NUMMER','ERSTKAUF'])
inx = pd.read_excel(inx_path,usecols=['NUMMER','NL_TYPE'])
rechnung = pd.read_csv(rech_f01,sep=sep,encoding=enc)

In [ ]:
## mapping the names to the codes in the columns related to last HJ and current HJ in the KW data
kw[last_hj] = kw[last_hj].map(kunden_segment_dict)
kw[current_hj] = kw[current_hj].map(kunden_segment_dict)

In [ ]:
## Data preprocessing to connect the clean tables together
kw['NUMMER'] = process_id(kw['NUMMER'])

adresse['NUMMER'] = process_id(adresse['NUMMER'])
adresse['GEBURT'] = process_date(adresse['GEBURT'])
adresse['SYS_ANLAGE'] = process_date(adresse['SYS_ANLAGE'])
adresse = assign_age(adresse)
adresse = assign_sources(adresse,'QUELLE')
adresse['QUELLE'] = adresse['SOURCE']
adresse = adresse.drop(columns=['SOURCE'])

stat['NUMMER'] = process_id(stat['NUMMER'])
stat['ERSTKAUF'] = process_date(stat['ERSTKAUF'])

inx['NUMMER'] = process_id(inx['NUMMER'])
rechnung['NUMMER'] = process_id(rechnung['NUMMER'])
rechnung['AUF_ANLAGE'] = process_date(rechnung['AUF_ANLAGE'])
rechnung = assign_sources(rechnung,'MEDIACODE')
rechnung['MEDIACODE'] = rechnung['SOURCE']
rechnung = rechnung.drop(columns=['SOURCE'])
rechnung['HERKUNFT'] = rechnung['HERKUNFT'].astype(str).str.replace('.0','')
rechnung['HERKUNFT'] = rechnung['HERKUNFT'].map(herkunft)
## Cleaning up the Versandkosten items from rechnungen, also dividing the JG and HG PREIS to separate columns for further processing
rechnung = rechnung[rechnung['WG_NAME'].str.contains(r'Versand',case=False,regex=True,na=False)==False]
rechnung.loc[rechnung['ART_NR'].str.contains(r'^\d+H[A-Z]\d+',case=False,na=False,regex=True),'PREIS_JG'] = rechnung.loc[rechnung['ART_NR'].str.contains(r'^\d+H[A-Z]\d+',case=False,na=False,regex=True),'PREIS']
rechnung.loc[rechnung['ART_NR'].str.contains(r'^\d+H[A-Z]\d+',case=False,na=False,regex=True)==False,'PREIS_HG'] = rechnung.loc[rechnung['ART_NR'].str.contains(r'^\d+H[A-Z]\d+',case=False,na=False,regex=True)==False,'PREIS']


In [ ]:
## Copying rechnung to filter it to the dates of the last HJ
rechnung_general = rechnung.copy()
rechnung_general = rechnung_general[(rechnung_general['AUF_ANLAGE']>= prev_start)&(rechnung_general['AUF_ANLAGE']<= prev_end)]
## Grouping the last HJ Rechnungen so we have certain values in cleanly named columns
rechnung_gr = rechnung_general.groupby(['NUMMER']).agg(             ANZ_AUF=('AUFTRAG_NR','nunique'),
                                                            AUF_ANLAGE=('AUF_ANLAGE','first'),
                                                            MEDIACODE=('MEDIACODE','first'),
                                                            UMSATZ_HG=('PREIS_HG','sum'),
                                                            UMSATZ_JG=('PREIS_JG','sum'),
                                                            HERKUNFT=('HERKUNFT','first')
                                                            ).reset_index()

In [ ]:
### Merging tables to each other using the customer ID (NUMMER) column
df = pd.merge(adresse,kw[['NUMMER',last_hj]],on='NUMMER',how='left')
df = pd.merge(df,stat,on='NUMMER',how='left')
df = pd.merge(df,inx,on='NUMMER',how='left')
df = pd.merge(df,rechnung_gr,on='NUMMER',how='left')


In [ ]:
### Cleaning up the df table, renaming the KW HJ column to Kundengruppe and removing duplicates from the data
df = df[['NUMMER', 'ANREDE', 'TITEL', 'VORNAME', 'NAME', 'QUELLE', 'LKZ', 'PLZ', 'ORT',
       'SYS_ANLAGE', 'AGE_GROUP',  last_hj, 'ERSTKAUF', 'NL_TYPE',
       'ANZ_AUF', 'HERKUNFT',  'MEDIACODE','AUF_ANLAGE', 'UMSATZ_HG','UMSATZ_JG']]
df = df.rename(columns={last_hj:'Kundengruppe'})
df = df.drop_duplicates(subset='NUMMER')

In [ ]:
### Defining interessenten customers, those who are in the system but have no orders
df.loc[(df['ERSTKAUF'].isna()),'Kundengruppe'] = "Interessenten"

In [ ]:
## filtering out the customers who have joined us in the current HJ
df = df[df['SYS_ANLAGE']<=prev_end]

# NK-1

In [ ]:
## Making the same grouping of the rechnung only for the current half year, so that we can use it for the Neukunden-1 group
rechnung_current = rechnung.copy()
rechnung_current = rechnung_current[(rechnung_current['AUF_ANLAGE']>= current_start)&(rechnung_current['AUF_ANLAGE']<=current_end)]

rechnung_nk_gr = rechnung_current.groupby(['NUMMER']).agg(  ANZ_AUF=('AUFTRAG_NR','nunique'),
                                                            AUF_ANLAGE=('AUF_ANLAGE','first'),
                                                            MEDIACODE=('MEDIACODE','first'),
                                                            UMSATZ_HG=('PREIS_HG','sum'),
                                                            UMSATZ_JG=('PREIS_JG','sum'),
                                                            HERKUNFT=('HERKUNFT','first')
                                                            ).reset_index()

## Selecting customers from the Adresse that are registered by us in the current halfyear
nk_adresse = adresse.copy()
nk_adresse = nk_adresse[nk_adresse['SYS_ANLAGE']>=current_start]

In [ ]:
## Connecting all data
nk_df = pd.merge(nk_adresse,kw[['NUMMER',current_hj]],on='NUMMER',how='left')
nk_df = pd.merge(nk_df,stat,on='NUMMER',how='left')
nk_df = pd.merge(nk_df,inx,on='NUMMER',how='left')
nk_df = pd.merge(nk_df,rechnung_nk_gr,on='NUMMER',how='left')

In [ ]:
### Cleaning up the df table, renaming the KW HJ column to Kundengruppe and removing duplicates from the data

nk_df = nk_df[['NUMMER', 'ANREDE', 'TITEL', 'VORNAME', 'NAME', 'QUELLE', 'LKZ', 'PLZ', 'ORT',
       'SYS_ANLAGE', 'AGE_GROUP',  current_hj, 'ERSTKAUF', 'NL_TYPE',
       'ANZ_AUF', 'HERKUNFT',  'MEDIACODE','AUF_ANLAGE', 'UMSATZ_HG','UMSATZ_JG']]
nk_df = nk_df.rename(columns={current_hj:'Kundengruppe'})
nk_df = nk_df.drop_duplicates(subset='NUMMER')
## Those with ERSTKAUF datum are neukunden-1 those without it are Interessenten
nk_df.loc[(nk_df['ERSTKAUF'].isna()),'Kundengruppe'] = "Interessenten"
nk_df.loc[(nk_df['ERSTKAUF'].notna()),'Kundengruppe'] = "Neukunden-1"


In [ ]:
## Concatenating the last hj customers with current hj customers
final_df = pd.concat([df,nk_df])
final_df = final_df.drop_duplicates(subset='NUMMER')
## Expanding the Neukunden-1 to those who are Interessenten but have an ERSTKAUF in the current hj
final_df.loc[(final_df['SYS_ANLAGE']<=prev_end)&(final_df['Kundengruppe']=='Interessenten')&(final_df['ERSTKAUF']>=current_start),'Kundengruppe'] = 'Neukunden-1'


In [ ]:
## Remove their Rechnungs data, and once again merge them with the rechnungs data so they don't have missing values
nk_final = final_df[final_df['Kundengruppe']=='Neukunden-1'][['NUMMER', 'ANREDE', 'TITEL', 'VORNAME', 'NAME', 'QUELLE', 'LKZ', 'PLZ',
       'ORT', 'SYS_ANLAGE', 'AGE_GROUP', 'Kundengruppe', 'ERSTKAUF', 'NL_TYPE']]
rechnung_nk_final_gr = rechnung.groupby(['NUMMER']).agg(  ANZ_AUF=('AUFTRAG_NR','nunique'),
                                                            AUF_ANLAGE=('AUF_ANLAGE','first'),
                                                            MEDIACODE=('MEDIACODE','first'),
                                                            UMSATZ_HG=('PREIS_HG','sum'),
                                                            UMSATZ_JG=('PREIS_JG','sum'),
                                                            HERKUNFT=('HERKUNFT','first')
                                                            ).reset_index()

## Merging the customers with the rechnungen
nk_final = nk_final.merge(rechnung_nk_final_gr,on='NUMMER',how='left')

In [ ]:
## removing this group from the final_Df and then adding the freshly created neukunden to the final_df_list
final_df = final_df[final_df['Kundengruppe'] != 'Neukunden-1']
final_df = pd.concat([final_df,nk_final])
## Removing duplicated Nummers
final_df = final_df.drop_duplicates(subset=['NUMMER'])

In [ ]:
## Removing Columns Anrede and Titel
final_df = final_df.drop(columns=['ANREDE','TITEL'])

In [ ]:
## Computing the Sum values for each kunden gruppe
gesamts = final_df.groupby('Kundengruppe').agg(GESAMT_ANZ_AUF=('ANZ_AUF','sum'),
                                     DURCHSCHNITT_ANZ_AUF=('ANZ_AUF','mean'),
                                     GESAMT_UMSATZ_HG=('UMSATZ_HG','sum'),
                                     GESAMT_UMSATZ_JG=('UMSATZ_JG','sum'),
                                     ).reset_index()

gesamts['GESAMT_UMSATZ'] = gesamts['GESAMT_UMSATZ_HG'] + gesamts['GESAMT_UMSATZ_JG']
gesamts['DURCHSCHNITT_UMSATZ'] = gesamts['GESAMT_UMSATZ'] / gesamts['GESAMT_ANZ_AUF']


In [ ]:
kg = list(final_df[final_df['Kundengruppe'].notna()]['Kundengruppe'].unique())
with pd.ExcelWriter('KundenInfo-2024-2HJ.xlsx',engine='xlsxwriter') as writer:
    gesamts.to_excel(writer,index=False,sheet_name='Gesamt Analytik')
    for item in kg:
        print(item)
        final_df[final_df['Kundengruppe'] == item].to_excel(writer,index=False,sheet_name=item)